Summary
===============
Starting with the DEM from the last post, we will first make some changes to the plotting, so that we can plot it without a border. After this, we can then add a coordinate system to the image, allowing it to be viewed in Google Earth.

Contour plots
===============

Our first step is to completely remove any border surrounding the plot. This can be acoumplished with the following [hack](http://stackoverflow.com/questions/9295026/matplotlib-plots-removing-axis-legends-and-white-spaces): 


In [6]:
import matplotlib.pyplot as plt

We can now proceeed to plot the DEM from the last post: 

In [4]:
fig = plt.figure()
fig.set_size_inches(10, 10)
ax = plt.Axes(fig, [0., 0., 1., 1.])
ax.set_axis_off()
fig.add_axes(ax)

plt.contour(DEM, levels,linewidths=0.2,colors='k')
plt.imshow(DEM,cmap ='RdYlGn_r',origin='lower')
plt.savefig('ContourMapTight.png',dpi=900)
plt.close()

NameError: name 'DEM' is not defined

GeoTIFFs
===============

[GeoTIFF](http://en.wikipedia.org/wiki/GeoTIFF) is a file format that allows spatial information to be embedded inside an TIFF file. Essentially, it allows you to store all the spatial information required to properly display the image. 


Our first step in generating a GeoTIFF is to load the .png created by Matplotlib as an numpy array called img. 

In [ ]:
img  = np.asarray(Image.open('ContourMapTight.png'))

I developed the boilderplate code from this [example](http://gis.stackexchange.com/questions/58517/python-gdal-save-array-as-raster-with-projection-from-other-file)

In [ ]:
def arrayToRaster(array,fileName,EPSGCode,xMin,xMax,yMin,yMax,numBands):
    xPixels = array.shape[1]  # number of pixels in x
    yPixels = array.shape[0]  # number of pixels in y
    pixelXSize =(xMax-xMin)/xPixels # size of the pixel in X direction     
    pixelYSize = -(yMax-yMin)/yPixels # size of the pixel in Y direction

    driver = gdal.GetDriverByName('GTiff')
    dataset = driver.Create(fileName,xPixels,yPixels,numBands,gdal.GDT_Byte, options = [ 'PHOTOMETRIC=RGB' ])
    dataset.SetGeoTransform((xMin,pixelXSize,0,yMax,0,pixelYSize))  

    datasetSRS = osr.SpatialReference()
    datasetSRS.ImportFromEPSG(EPSGCode)
    dataset.SetProjection(datasetSRS.ExportToWkt())
    
    for i in range(0,numBands):
        dataset.GetRasterBand(i+1).WriteArray(array[:,:,i])

    dataset.FlushCache()  # Write to disk.

EPSG code 4326 is the WGS84 projection.

In [ ]:
EPSGCode = 4326
numBands = 3
arrayToRaster(img,'OUT.TIF',EPSGCode,np.min(xArray),np.max(xArray),np.min(yArray),np.max(yArray),numBands)

Finally, we are left with a TIFF file called "OUT.TIFF" which we can open in Google Earth. Simply go to File -> Import, and open the GeoTIFF. 